In [1]:
import sys
import os
sys.path.append('/Users/adam2392/Documents/tvb/')
sys.path.append('/Users/adam2392/Documents/tvb/_tvbdata/')
sys.path.append('/Users/adam2392/Documents/tvb/_tvblibrary/')
from tvb.simulator.lab import *
import numpy as np
import pandas as pd
import scipy
from sklearn import cluster

import tvbsim
from runmainsim import *

%load_ext autoreload
%autoreload 2

# Data Input

Set up where we get data from. This is the data computed on all the tvb forwardsims, with the FFT. We grab the data here and load it locally onto RAM

In [3]:
MOVEDCONTACTS = 1

if MOVEDCONTACTS:
    datadir = '/home/adamli/data/output/frequencyanalysis/moved/'
else:
    datadir = '/home/adamli/data/output/frequencyanalysis/notmoved/'
if MOVEDCONTACTS:
    datadir = '/Volumes/ADAM LI/pydata/output/frequencyanalysis/moved/'
else:
    datadir = '/Volumes/ADAM LI/pydata/output/frequencyanalysis/notmoved/'

metadatadir = '/Volumes/ADAM LI/pydata/metadata/'    

datafiles = os.listdir(datadir)
print('Moving contacts: ', MOVEDCONTACTS)
print(datafiles)

('Moving contacts: ', 1)
['id014_rb_fourier_sim_nez1_npz1.npz', 'id002_cj_fourier_sim_nez1_npz1.npz', 'id001_ac_fourier_sim_nez1_npz1.npz']


# Setup Processing Pipeline

Setup a processing pipeline and procedure for putting data into a processed format ready for DNN training.

For all DNN we need:
* timepoints of windows
* frequency bands of activity
* seizure onsets offsets indices
* simulation parameters (x0 vals, ks, ez/pz regions, etc.)
* FFT parameters

In [4]:
# get factors of the number of regions
def get_factors(x):
   # This function takes a number and prints the factors

    factors = []
    for i in range(1, x + 1):
        if x % i == 0:
            factors.append(i)
    return factors

# establish frequency bands
freqbands = {
#     'deltatheta': [0,8],
#     'alpha': [8, 16],
#     'beta': [16, 30],
#     'gamma': [30,90],
#     'highgamma': [90, 500]
    'lowfreq': [0, 16],
    'midfreq': [16, 33],
    'gamma': [33, 90],
    'highgamma': [90, 501],
}

data = np.load(os.path.join(datadir,datafiles[0]))
# extract data from the numpy file
print datafiles[0]
freqs = data['freqs']
metadata = data['metadata'].item()

freqbandindices = {}
for band in freqbands:
    lowerband = freqbands[band][0]
    upperband = freqbands[band][1]
    
    # get indices where the freq bands are put in
    freqbandindices[band] = np.where((freqs >= lowerband) & (freqs < upperband))
    freqbandindices[band] = [freqbandindices[band][0][0], freqbandindices[band][0][-1]]

print metadata.keys()
print freqbands
print freqs
print freqbandindices

id014_rb_fourier_sim_nez1_npz1.npz
['x0norm', 'patient', 'seeg_contacts', 'pz', 'ez', 'seeg_xyz', 'offsettimes', 'regions_centers', 'regions', 'stepsize', 'winsize', 'x0pz', 'onsettimes', 'pzindices', 'x0ez', 'ezindices', 'mtbandwidth']
{'highgamma': [90, 501], 'lowfreq': [0, 16], 'gamma': [33, 90], 'midfreq': [16, 33]}
[   0.    4.    8.   12.   16.   20.   24.   28.   32.   36.   40.   44.
   48.   52.   56.   60.   64.   68.   72.   76.   80.   84.   88.   92.
   96.  100.  104.  108.  112.  116.  120.  124.  128.  132.  136.  140.
  144.  148.  152.  156.  160.  164.  168.  172.  176.  180.  184.  188.
  192.  196.  200.  204.  208.  212.  216.  220.  224.  228.  232.  236.
  240.  244.  248.  252.  256.  260.  264.  268.  272.  276.  280.  284.
  288.  292.  296.  300.  304.  308.  312.  316.  320.  324.  328.  332.
  336.  340.  344.  348.  352.  356.  360.  364.  368.  372.  376.  380.
  384.  388.  392.  396.  400.  404.  408.  412.  416.  420.  424.  428.
  432.  436.  440.  4

In [5]:
outputdir = '/Volumes/ADAM LI/pydata/output/frequencyanalysis/summarized/'

powerbands = {}
for idx,datafile in enumerate(datafiles):
    print os.path.join(datadir,datafile)
    data = np.load(os.path.join(datadir,datafile))
    
    # extract data from the numpy file
    power = data['power']
    freqs = data['freqs']
    timepoints = data['timepoints']
    metadata = data['metadata'].item()
    metadata['freqbands'] = freqbands
    
    # compress data using frequency bands
    for band in freqbandindices:
        indices = freqbandindices[band]
        # average between these two indices
        buff = np.mean(power[:,indices[0]:indices[1]+1,:], axis=1)[np.newaxis,:,:]
        
        # project onto a 
        print buff.shape
        if band in powerbands:
            powerbands[band] = np.concatenate((powerbands[band], buff[np.newaxis,:,:]),axis=0)
        else:
            powerbands[band] = buff[np.newaxis,:,:]
        
# store all data 
# - power, timepoints, metadata
print data.keys()
print metadata.keys()
print power.shape

for band in powerbands:
    print powerbands[band].shape

/Volumes/ADAM LI/pydata/output/frequencyanalysis/moved/id014_rb_fourier_sim_nez1_npz1.npz
(1, 165, 1919)
(1, 165, 1919)
(1, 165, 1919)
(1, 165, 1919)
/Volumes/ADAM LI/pydata/output/frequencyanalysis/moved/id002_cj_fourier_sim_nez1_npz1.npz
(1, 162, 1919)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [7]:
print metadata.keys()
print metadata['onsettimes']
print metadata['offsettimes']

onsettimes = metadata['onsettimes']
offsettimes = metadata['offsettimes']

if onsettimes[0] > offsettimes[0]:
    onsettimes = metadata['offsettimes']
    offsettimes = metadata['onsettimes']


['x0norm', 'freqbands', 'patient', 'seeg_contacts', 'pz', 'ez', 'seeg_xyz', 'offsettimes', 'regions_centers', 'regions', 'stepsize', 'winsize', 'x0pz', 'onsettimes', 'pzindices', 'x0ez', 'ezindices', 'mtbandwidth']
(27173, 61757, 91776, 116858, 145280, 181071, 215871)
(12535, 50153, 82738, 111110, 131869, 169384, 201387)


# Extract Power Data and Condense By Frequency Band

In [24]:
patient = 'id014_rb'
datafiles = [i for i in os.listdir(datadir) if patient in i]
print datafiles
datafile = datafiles[0]

data = np.load(os.path.join(datadir,datafile))
    
# extract data from the numpy file
power = data['power']
freqs = data['freqs']
timepoints = data['timepoints']
metadata = data['metadata'].item()
metadata['freqbands'] = freqbands

powerbands = {}
# compress data using frequency bands
for band in freqbandindices:
    indices = freqbandindices[band]
    # average between these two indices
    powerbands[band] = np.mean(power[:,indices[0]:indices[1]+1,:], axis=1) #[np.newaxis,:,:]

    # project onto a 
#     print buff.shape
#     if band in powerbands:
#         powerbands[band] = np.concatenate((powerbands[band], buff),axis=0)
#     else:
#         powerbands[band] = buff
        
for band in freqbandindices:
    print powerbands[band].shape
print powerbands.keys()

['id014_rb_fourier_sim_nez1_npz1.npz']
(165, 1919)
(165, 1919)
(165, 1919)
(165, 1919)
['highgamma', 'lowfreq', 'gamma', 'midfreq']


# Use Gain Matrix From TVB
Using gain matrix from TVb, we just flatten the regions after projecting the power of each SEEG channel onto that region.

1. Project frequency band power onto regions using gain matrix for each time window
2. Arrange vector per time window into 2D matrix to get 2D matrices over time of averaged power from each frequency band
3. Concatenate into a condensed matrix
4. save using np.savez_compressed, or np.save

**Note: Perhaps I should save the gain matrix from all tvb forward simulations from now on and pass it through to any analysis that needs it -> frequency analysis.

### Need to add code to keep track of all the different onset/offsets

In [10]:
# RECOMPUTE GAIN MATRIX USING THIS PATIENT
project_dir = os.path.join('/Volumes/ADAM LI/pydata/metadata/', patient)
use_subcort = True

# load in the vertices, normals and areas of gain matrix
verts, normals, areas, regmap = tvbsim.util.read_surf(project_dir, use_subcort)
                             
confile = os.path.join(metadatadir, patient, "connectivity.zip")
sensorsfile = os.path.join(metadatadir, patient, "seeg.txt")
####################### 1. Structural Connectivity ########################
con = tvbsim.initializers.connectivity.initconn(confile)
seeg_xyz = metadata['seeg_xyz']
seeg_labels = metadata['seeg_contacts']

# extract the seeg_xyz coords and the region centers
region_centers = con.centres
regions = con.region_labels

# get the ez regions
ezregion = metadata['ez']
ezindices = metadata['ezindices']

print seeg_xyz.shape
print ezregion
print ezindices

(165, 3)
['ctx-lh-bankssts']
[0]


In [11]:
# reshape the regions of 84 into a parcellated rectangular "image"
# height = np.ce
factors = get_factors(84)
height = factors[len(factors)/2]
width = len(regions) / height

# shapes of a new region/region centers
new_region_centers = np.reshape(region_centers, (height, width, 3), order='C')
new_regions = np.reshape(regions, (height,width), order='C')

print "Dimensions of image are: "
print height
print width
print new_region_centers.shape

# check seeg_xyz moved correctly - In early simulation data results, was not correct
buff = seeg_xyz - region_centers[:, np.newaxis]
buff = np.sqrt((buff**2).sum(axis=-1))
test = np.where(buff==0)
indice = test[1]

if not indice:
    print "Need to recompute the seeg_xyz and save correctly"

In [12]:
modgain = tvbsim.util.gain_matrix_inv_square(verts, areas,
                            regmap, len(regions), seeg_xyz)
modgain = modgain.T
print modgain.shape

(84, 165)


In [29]:
# map seeg activity -> epileptor source and create data structure
for idx,band in enumerate(powerbands):
    mapped_power_band = np.tensordot(modgain, powerbands[band], axes=([1],[0]))
    
    # reformat into an image along the region axes
#     powerbands[band] = mapped_power_band.reshape(height, width, mapped_power_band.shape[1], 
#                                                      order='C')
    
    if idx==0:
        mapped_power_bands = mapped_power_band.reshape(height, width, mapped_power_band.shape[1], 
                                                     order='C')[np.newaxis,:,:,:]
    else:
        mapped_power_bands = np.append(mapped_power_bands, 
                    mapped_power_band.reshape(height, width, mapped_power_band.shape[1], 
                                                     order='C')[np.newaxis,:,:,:],
                                      axis=0)
    print powerbands[band].shape
    
# new condensed data structure is H x W x F x T, to concatenate more, add to T dimension
mapped_power_bands = mapped_power_bands.swapaxes(0,2)
print mapped_power_bands.shape

(165, 1919)
(165, 1919)
(165, 1919)
(165, 1919)
(7, 12, 4, 1919)


# Use K Nearest Neighbors

Get k nearest neighbors and project power activity into those neighbors.

In [45]:
# map seeg_xyz to 3 closest region_centers
tree = scipy.spatial.KDTree(region_centers)
seeg_near_indices = []

seeg_counter = np.zeros(len(regions))

seeg_mapped_power = np.zeros((len(regions), len(freqbands), powerbands[band].shape[1]))

for ichan in range(0, len(seeg_labels)):
    near_regions = tree.query(seeg_xyz[ichan,:].squeeze(), k=3)
    near_indices = near_regions[1]
    
    # go through each frequency band and map activity onto those near indices
    for idx,band in enumerate(powerbands):
        chanpower = powerbands[band][ichan,:]
        seeg_mapped_power[near_indices,idx,:] += chanpower.astype('float64')
    
    seeg_counter[near_indices] += 1
    seeg_near_indices.append(near_indices)

# get the average based on how many contributions of the seeg power was to this region
seeg_mapped_power = seeg_mapped_power / seeg_counter[:,np.newaxis,np.newaxis]

print near_regions
print near_indices
print len(seeg_near_indices)
print seeg_mapped_power.shape
# print seeg_counter

(array([ 27.66716676,  28.02515861,  28.44087267]), array([13, 31, 28]))
[13 31 28]
165
(84, 4, 1919)


In [65]:
seeg_mapped_power = seeg_mapped_power.reshape(height, width, len(freqbands), powerbands[band].shape[1], 
                                                     order='C')
seeg_mapped_power = seeg_mapped_power.swapaxes(0,1)
print seeg_mapped_power.shape

(7, 12, 4, 1919)


# Fall Off with Modified 1/r^2 Function

For any distances 0 <= r <= 1, set gain to 1, and then set gain as a function of 1/r^2.

#### Note: We should check that the units of them are meters and not mm? Is the law only apply for metric units?

In [110]:
# map seeg_xyz to the rest of the regions from a factor of 
dr = region_centers - seeg_xyz[:,np.newaxis] # computes distance along each axis
ndr = np.sqrt((dr**2).sum(axis=-1)) # computes euclidean distance
Vr = 1/(ndr**2) # fall off as a function of r^2

inf_indices = np.where(np.isinf(Vr))
small_indices = np.where(ndr <= 1)

# can either set to 1, or the max that there currently is + some small epsilon
# the problem with setting to 1 is that the signal might drown out everything else
Vr[small_indices] = np.nanmax(np.ma.masked_invalid(Vr[:])) + np.nanmin(Vr[:])

# normalize Vr with minmax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
Vr = scaler.fit_transform(Vr)

# store the transpose
Vr = Vr.T

print Vr.shape
# print Vr[small_indices]
# print inf_indices
# print small_indices
# print np.where(np.isinf(Vr))


# import matplotlib.pyplot as plt
# plt.imshow(Vr, aspect='auto')
# plt.colorbar()
# plt.show()

(84, 165)


In [111]:
# map seeg activity -> epileptor source and create data structure
for idx,band in enumerate(powerbands):
    mapped_power_band = np.tensordot(Vr, powerbands[band], axes=([1],[0]))
    
    # store the formatted power bands
    if idx==0:
        mapped_power_bands = mapped_power_band.reshape(height, width, mapped_power_band.shape[1], 
                                                     order='C')[np.newaxis,:,:,:]
    else:
        mapped_power_bands = np.append(mapped_power_bands, 
                    mapped_power_band.reshape(height, width, mapped_power_band.shape[1], 
                                                     order='C')[np.newaxis,:,:,:],
                                      axis=0)
    print powerbands[band].shape
    
# new condensed data structure is H x W x F x T, to concatenate more, add to T dimension
mapped_power_bands = mapped_power_bands.swapaxes(0,2)
print mapped_power_bands.shape

(165, 1919)
(165, 1919)
(165, 1919)
(165, 1919)
(7, 12, 4, 1919)


# Use PCA with K Nearest Neighbors

Using PCA to project everything SEEG and regions xyz onto 2D coordinates and then perform K nearest neighbors to average power activity for each frequency band into that slot.

# Lastly: 3D Convolutional Neural Network

Instead of using a traditional 2D convolutional neural network, we can perhaps define a cube around each brain, and perform 3D sparse convolutional neural network training also to compare.

This effort might be worse because of the issues of sparsity and the implementability on a GPU with that amount of data.

# Old Notes and Tries

Different things I was trying to script...

In [ ]:
patient = 'id014_rb'
confile = os.path.join(metadatadir, patient, "connectivity.zip")
sensorsfile = os.path.join(metadatadir, patient, "seeg.txt")
####################### 1. Structural Connectivity ########################
con = tvbsim.initializers.connectivity.initconn(confile)
seeg_xyz = metadata['seeg_xyz']
seeg_labels = metadata['seeg_contacts']

# extract the seeg_xyz coords and the region centers
# seeg_df = tvbsim.util.extractseegxyz(sensorsfile)
# seeg_labels = seeg_df.index.values
# seeg_xyz = seeg_df.values

# extract the seeg_xyz coords and the region centers
region_centers = con.centres
regions = con.region_labels

print regions.shape
print region_centers.shape
print seeg_xyz.shape
print seeg_labels.shape

test = seeg_xyz - region_centers[:, np.newaxis]
test = np.sqrt((test**2).sum(axis=-1))
print "looking at test"
print test.shape
print np.where(test==0)

print test[0,133]
print regions[0]

########## MOVE INDICES
# ezindices = metadata['ezindices']
# # initialize object to assist in moving seeg contacts
# movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
#                                    regions, region_centers, True)
# # move electrodes onto ez indices
# elecmovedindices = []
# for ezindex in ezindices:
#     print "Moving onto current ez index: ", ezindex, " at ", regions[ezindex]
#      # find the closest contact index and distance
#     seeg_index, distance = movecontact.findclosestcontact(ezindex, elecmovedindices)

#     # get the modified seeg xyz and gain matrix
#     modseeg, electrodeindices = movecontact.movecontact(ezindex, seeg_index)
#     elecmovedindices.append(electrodeindices)
    
# print modseeg[elecmovedindices]
# print region_centers[ezindices]
# test = modseeg[elecmovedindices] - region_centers[ezindices, np.newaxis]
# print "looking at test"
# print test
# print test.shape
# buff = np.sqrt((test**2).sum(axis=-1))
# print buff
# print buff.shape

# test = 1/(4*np.pi*buff)
# print test